### Import Libraries

In [4]:
import surgeo as sg
import pandas as pd
import numpy as np
import glob as gb
import os
import arcpy

### Set Path

In [2]:
path = "C:\\Users\\padu\\Dropbox (UNC Charlotte)\\NewMovers\\Movers2020_2022"
os.chdir(path)

In [3]:
ProjectGDB =  r"C:\\Users\\padu\\Dropbox (UNC Charlotte)\\NewMovers\\Movers2020_2022\\MoversReview\\MoversReview.gdb"
arcpy.env.overwriteOutput = True
arcpy.env.workspace = ProjectGDB

### Merge Individual Files

In [4]:
movers = [row for row in gb.glob("*.csv")]
len(movers)

In [5]:
Allmovers =  pd.concat([pd.read_csv(row) for row in movers])
Allmovers.shape

In [6]:
Allmovers = Allmovers.drop_duplicates()
Allmovers.shape

In [5]:
path = 'C:\\Users\\padu\Dropbox (UNC Charlotte)\\NewMovers\\FinalMovers'
os.chdir(path)

In [8]:
Allmovers.to_csv("Movers2022.csv")

In [6]:
#Read Movers Data

data = pd.read_csv('Movers2022.csv')
data

C:\Users\padu\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,OID_,Title,First_Name,Middle_Initial,Last_Name,Suffix,House_Number,Street_Pre_Directional,Street_Name,Street_Suffix,...,Estimated_Household_Income,Estimated_Home_Value,Estimated_Age,Move_Distance,Record_ID,Census_Block_Group,ZIP,PO_NAME,POPULATION,SQMI
0,1,NaN,Taniya,NaN,Sachdeva,NaN,1428.0,NaN,Winding Stream,Rd,...,"$30,000 - $39,999","$125,000 - $149,999",40-44,4 miles,343049573,2,28206,Charlotte,16184,7.14
1,2,NaN,Darren,NaN,Tyrrell,NaN,210.0,NaN,Alpha Mill,Ln,...,"Under $20,000",NaN,35-39,NaN,343049574,2,28206,Charlotte,16184,7.14
2,3,NaN,Adrian,NaN,Caspar,NaN,1316.0,N,Davidson,St,...,"$50,000 - $59,999","$200,000 - $249,999",30-34,1 miles,343049575,1,28206,Charlotte,16184,7.14
3,4,NaN,Mirlourde,NaN,Petit-Frere,NaN,1018.0,N,Caldwell,St,...,"Under $20,000",NaN,40-44,4 miles,343049576,2,28206,Charlotte,16184,7.14
4,5,NaN,Marissa,NaN,Butts,NaN,230.0,NaN,Alpha Mill,Ln,...,"Under $20,000",NaN,35-39,4 miles,343049577,2,28206,Charlotte,16184,7.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97643,97644,NaN,Sylvia,NaN,Ruiz,NaN,2424.0,NaN,Rachel,St,...,"Under $20,000",NaN,55-59,139 miles,343049568,2,28206,Charlotte,16184,7.14
97644,97645,NaN,Dawanas,NaN,King,NaN,1018.0,N,Caldwell,St,...,"Under $20,000",NaN,40-44,122 miles,343049569,2,28206,Charlotte,16184,7.14
97645,97646,NaN,Kathryn,NaN,Gilbert,NaN,1700.0,N,Brevard,St,...,"$30,000 - $39,999",NaN,30-34,16 miles,343049570,1,28206,Charlotte,16184,7.14
97646,97647,NaN,Bart,NaN,Anthony,NaN,1700.0,N,Brevard,St,...,"$30,000 - $39,999",NaN,30-34,1 miles,343049571,1,28206,Charlotte,16184,7.14


- Convert all strings to upper case

In [11]:
data['First_Name'] = [str(row).upper() for row in data['First_Name']]
data['Last_Name'] = [str(row).upper() for row in data['Last_Name']]
data['ZIP'] = [str(row) for row in data['ZIP']]


- Create an ID with names and zipcodes 

In [12]:
data['ID'] = data['First_Name'] + data['Last_Name'] + data['ZIP']

- Initiate BIGS models

In [5]:
G = sg.BIFSGModel()
S = sg.SurnameModel()

- Get racial probabilities based on name and race

In [9]:

race = G.get_probabilities(data['First_Name'], data['Last_Name'],data['ZIP'])
racesur = G.get_probabilities(data['First_Name'], data['Last_Name'],data['ZIP'])
race.columns

Index(['zcta5', 'first_name', 'surname', 'white', 'black', 'api', 'native',
       'multiple', 'hispanic'],
      dtype='object')

- For each row, return the return a race based on the highest probability assigned

In [11]:
racesur['Race'] =racesur[['white', 'black', 'api', 'native','multiple', 'hispanic']].idxmax(axis =1)
racesur

,zcta5,first_name,surname,white,black,api,native,multiple,hispanic,Race
0,28206,TANIYA,SACHDEVA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28206,DARREN,TYRRELL,0.648170,0.302097,0.001792,0.006866,0.036260,0.004815,white
2,28206,ADRIAN,CASPAR,0.324021,0.252778,0.001456,0.013598,0.026248,0.381898,hispanic
3,28206,MIRLOURDE,PETITFRERE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28206,MARISSA,BUTTS,0.119530,0.840483,0.000881,0.000000,0.026185,0.012922,black
...,...,...,...,...,...,...,...,...,...,...
97643,28206,SYLVIA,RUIZ,0.005476,0.043645,0.001071,0.001076,0.000453,0.948278,hispanic
97644,28206,DAWANAS,KING,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97645,28206,KATHRYN,GILBERT,0.494555,0.477608,0.000258,0.001385,0.021735,0.004459,white
97646,28206,BART,ANTHONY,0.356803,0.637413,0.001826,0.000000,0.000000,0.003958,black


In [14]:
race['Race'] = race[['white', 'black', 'api', 'native','multiple', 'hispanic']].idxmax(axis =1)
race

,zcta5,first_name,surname,white,black,api,native,multiple,hispanic,Race
0,28206,TANIYA,SACHDEVA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28206,DARREN,TYRRELL,0.648170,0.302097,0.001792,0.006866,0.036260,0.004815,white
2,28206,ADRIAN,CASPAR,0.324021,0.252778,0.001456,0.013598,0.026248,0.381898,hispanic
3,28206,MIRLOURDE,PETITFRERE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28206,MARISSA,BUTTS,0.119530,0.840483,0.000881,0.000000,0.026185,0.012922,black
...,...,...,...,...,...,...,...,...,...,...
97643,28206,SYLVIA,RUIZ,0.005476,0.043645,0.001071,0.001076,0.000453,0.948278,hispanic
97644,28206,DAWANAS,KING,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97645,28206,KATHRYN,GILBERT,0.494555,0.477608,0.000258,0.001385,0.021735,0.004459,white
97646,28206,BART,ANTHONY,0.356803,0.637413,0.001826,0.000000,0.000000,0.003958,black


- Join data to zip codes for mapping

In [15]:
race['zcta5'] = [str(row) for row in race['zcta5']]
race['ID'] = race['first_name'] + race['surname'] + race['zcta5']

In [16]:
TotalID = pd.merge(data, race, on='ID', how='left')
TotalID.shape

(102318, 49)

In [18]:
TotalID = TotalID.drop_duplicates()
# TotalID.to_csv('FinalMovers.csv')
TotalID.shape

(97648, 49)